<a href="https://colab.research.google.com/github/F1ameX/TUSUR_ML/blob/main/TUSUR_HW_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Первый вариант



**Задача**: Использовать предобученную языковую модель для генерации продолжения текста.

В рамках задачи необходимо использовать библиотеку transformers и небольшую GPT-модель для генерации продолжения текста с заданными параметрами. Если модель сгенерировала неполное предложение (без точки/восклицательного знака/знака вопроса/многоточия), то удалить эту часть и оставить только полные предложения. Затем убрать параметр repetition_penalty и сравнить качество генерации.

## Установка и импорт необходимых библиотек

In [1]:
!pip install --quiet transformers

In [3]:
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

## Загрузка модели и токенизатора

In [4]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

## Формирование незаконченных предложений разной длины

Для эксперимента проведём анализ по трём разным незаконченным предложениям: длинной, средней и короткой длины соответственно. Идея в том, что предложения, которые практически завершены, модель будет продолжать чаще одинаково, чем в тех, где есть больше места "разгуляться".

In [5]:
long_unfinished = ['Возьму с собой зонт, потому что завтра будет плохая',
                   'Он приехал на дорогой машине, возможно, у него хорошая',
                   'Завтра я собираюсь встретиться с друзьями, потому что важно поддерживать',
                   'Вчера мы посетили новый ресторан, и я до сих пор думаю, что еда была просто',
                   'Наша команда подготовила доклад для конференции, осталось лишь']

middle_unfinished = ['Она купила новый телефон, который',
                     'Мы планируем поехать в отпуск, но',
                     'Сегодня на работе было довольно спокойно, хотя',
                     'Я пытался разобраться в этом вопросе, но',
                     'Он давно не появлялся в нашем офисе, все']

short_unfinished = ['Он сказал, что',
                    'Я думаю, что',
                    'Она не ответила на',
                    'Мне нужно сделать это',
                    'Ты не можешь объяснить']

## Вспомогательные функции

In [12]:
def tokenize(sentences):
  return tokenizer(sentences, padding = True, truncation = True, return_tensors = 'pt')


def generate_sentences(sentences):
  penalty = model.generate(
    input_ids = tokenize(sentences)['input_ids'],
    max_length = 150,
    repetition_penalty = 1.2)

  no_penalty = model.generate(
    input_ids = tokenize(sentences)['input_ids'],
    max_length = 150)

  return penalty, no_penalty


def preprocess_result(full_sentences):
  for idx in range(5):
    result = tokenizer.decode(full_sentences[idx], skip_special_tokens = True)
    result = re.split(r'[.!?]', result)[0].replace('\n', '')
    print(result)
  print('=' * 100)


## Инференс модели

Сгенерируем продолжение всех трёх видов предложений.

In [13]:
long_with_penalty, long_without_penalty = generate_sentences(long_unfinished)
middle_with_penalty, middle_without_penalty = generate_sentences(middle_unfinished)
short_with_penalty, short_without_penalty = generate_sentences(short_unfinished)

In [14]:
print('Длинные предложения с ограничением repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(long_with_penalty)
print('Длинные предложения без ограничения repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(long_without_penalty)

Длинные предложения с ограничением repetition_penalty = 1.2
Возьму с собой зонт, потому что завтра будет плохая погода
Он приехал на дорогой машине, возможно, у него хорошая машина
Завтра я собираюсь встретиться с друзьями, потому что важно поддерживать отношения
Вчера мы посетили новый ресторан, и я до сих пор думаю, что еда была просто великолепна
Наша команда подготовила доклад для конференции, осталось лишь подготовить его к печати
Длинные предложения без ограничения repetition_penalty = 1.2
Возьму с собой зонт, потому что завтра будет плохая погода
Он приехал на дорогой машине, возможно, у него хорошая машина
Завтра я собираюсь встретиться с друзьями, потому что важно поддерживать отношения с людьми, которые мне дороги
Вчера мы посетили новый ресторан, и я до сих пор думаю, что еда была просто великолепна
Наша команда подготовила доклад для конференции, осталось лишь подготовить его к печати


In [15]:
print('Средние предложения с ограничением repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(middle_with_penalty)
print('Средние предложения без ограничения repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(middle_without_penalty)

Средние предложения с ограничением repetition_penalty = 1.2
Она купила новый телефон, который ей очень нравился
Мы планируем поехать в отпуск, но не знаем с чего начать
Сегодня на работе было довольно спокойно, хотя и не очень хорошо
Я пытался разобраться в этом вопросе, но не смог
Он давно не появлялся в нашем офисе, все время пропадал где-то
Средние предложения без ограничения repetition_penalty = 1.2
Она купила новый телефон, который ей очень нравился
Мы планируем поехать в отпуск, но не знаем, как это сделать
Сегодня на работе было довольно спокойно, хотя и не очень хорошо
Я пытался разобраться в этом вопросе, но не смог
Он давно не появлялся в нашем офисе, все время пропадал в офисе, а теперь вот снова появился


In [16]:
print('Короткие предложения с ограничением repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(short_with_penalty)
print('Короткие предложения без ограничения repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(short_without_penalty)

Короткие предложения с ограничением repetition_penalty = 1.2
Он сказал, что он не может быть в этом уверен
Я думаю, что это не так
Она не ответила на его вопрос
Мне нужно сделать это, — сказал он
Ты не можешь объяснить, почему ты так поступил
Короткие предложения без ограничения repetition_penalty = 1.2
Он сказал, что он не может быть в этом уверен
Я думаю, что это не так
Она не ответила на его вопрос
Мне нужно сделать это, — сказал он
Ты не можешь объяснить, почему ты не можешь быть с ним


## Вывод

Эксперимент показал, что результаты генерации предложений не сильно различались, несмотря на изменение длины исходных фраз и настройки параметра repetition_penalty. Модель продолжала генерировать тексты схожим образом независимо от того, было ли начальное предложение коротким, средним или длинным. Это может свидетельствовать о том, что модель не проявляет значительной зависимости от длины начального контекста и параметра, предназначенного для снижения повторений, при условии, что контекст достаточно ясен. Вполне возможно, что модель больше ориентируется на структуру фразы и семантические связи, чем на продолжительность начального текста.

# Второй вариант

**Задача**: Использовать предобученную языковую модель для оценки тональности текста.

В рамках задачи необходимо использовать библиотеку transformers и небольшую BERT-модель для определения степени токсичности высказывания.

1. Загрузить библиотеку transformers
2. Загрузить модель и токенизатор для модели [s-nlp/russian_toxicity_classifier](https://huggingface.co/s-nlp/russian_toxicity_classifier)
3. Написать в сумме 10 предложений, где присутствуют положительные, нейтральные и негативные высказывания.
4. Получить предсказания и вывести их в формате "Текст: класс: оценка"
5. Описать шаги для получения предсказаний.
6. Проанализировать результаты и указать, имеет ли модель склонность оценивать нейтральные комментарии ближе к отрицательным высказываниям. Вывод должен быть сформулирован в отдельной ячейке.